In [45]:
%reload_ext autoreload
%autoreload 2

from IPython.core.display import Markdown
from tqdm.auto import tqdm
import config
import docs
import pandas as pd
import requests
import db
from datetime import datetime

pd.set_option('display.max_colwidth', None)

INDICATOR = 'bike_lanes'
CONFIG = config.get_config(INDICATOR, '../config.toml')

display(Markdown('## Raw data path'))
raw_dir_path = str(CONFIG['raw_dir']).replace('\\', '/')
display(Markdown(f"[{raw_dir_path}]({raw_dir_path})"))


## Raw data path

[C:/Users/tan/src/regional-pm-2023/data/raw/transportation/active/bike_lanes](C:/Users/tan/src/regional-pm-2023/data/raw/transportation/active/bike_lanes)

# Transportation Planning: Active Transportation

## Bike Lane Miles

In [46]:
docs.describe_indicator(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

Length of bike lanes.

nan

In [47]:
docs.list_schema(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,name,description,type
column,,,
year,Year,Year of record.,datetime
lane_miles,Lane Miles,Length (in miles) of all bike lanes in a given year.,float


In [48]:
docs.list_sources(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,name,organization,active,notes
source,,,,
bike_lane_db,Bike Lane GIS Tables,SANDAG,True,"Geometry for every few years, none of them have consistent set of fields."


In [49]:
steps = docs.list_update_steps(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)
steps

,step
0,Extract each geometry vintage from SANDAG's SQL Server.
1,Calculate bike miles from each separate geometry vintage and combine together.


In [50]:
docs.list_remarks(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,author,note
0,TAN,NaN
1,TAN,NaN


### Step 0: Download Vintages

In [51]:
display(steps.loc[0])

step    Extract each geometry vintage from SANDAG's SQL Server.
Name: 0, dtype: object

In [52]:
con = db.get_db_connection(
    'SQL2014b8',
    'GeoDepot',
)

In [53]:
bike_lanes_2021 = (
    pd.read_sql(
        """--sql
        SELECT
            [length_miles]
        FROM [gis].[BIKEROUTES]
        WHERE [Route_Class_Name] = 'Bike Lane'
        """,
        con=con
    )
    .assign(year=datetime(2021, 1, 1))
    .set_index('year')
)
display(bike_lanes_2021.tail(3))
bike_lanes_2021.sum()

,length_miles
year,
2021-01-01,1.013041
2021-01-01,0.241159
2021-01-01,7.399292


length_miles    1059.190394
dtype: float64

Note that `[BIKEROUTES2016]` did not have a length field, and because of elevation, I wasn't sure if I can technically calculate it.

For `[BIKEWOUTES2014]`, distance was given in feet so it had to be adjusted to miles. I found that bike lanes are FClass 2 from the documentation.

In [54]:
bike_lanes_2014 = (
    pd.read_sql(
        """--sql
        SELECT
            [Distance]/5280 AS [length_miles]
        FROM [gis].[BIKEROUTES2014]
        WHERE [Func_Class] = 2
        """,
        con=con
    )
    .assign(year=datetime(2014, 1, 1))
    .set_index('year')
)
display(bike_lanes_2014.tail(3))
bike_lanes_2014.sum()

,length_miles
year,
2014-01-01,0.005161
2014-01-01,0.052560
2014-01-01,0.328530


length_miles    163.343666
dtype: float64

`[BIKEROUTES2009]` also does not have a lenth field, could we calculate accurately from distances without height data?

### Step 1: Calculate and combine.

In [55]:
display(steps.loc[1])

step    Calculate bike miles from each separate geometry vintage and combine together.
Name: 1, dtype: object

In [62]:
bike_lane_miles = (
    pd.concat(
        [
            bike_lanes_2014,
            bike_lanes_2021,
        ]
    )
    .groupby('year')
    .sum()
)
bike_lane_miles

,length_miles
year,
2014-01-01,163.343666
2021-01-01,1059.190394


### Save Data

In [56]:
display(Markdown('#### Clean data path'))
clean_dir_path = str(CONFIG['clean_dir']).replace('\\', '/')
display(Markdown(f"[{clean_dir_path}]({clean_dir_path})"))

#### Clean data path

[C:/Users/tan/src/regional-pm-2023/data/clean/transportation/active/bike_lanes](C:/Users/tan/src/regional-pm-2023/data/clean/transportation/active/bike_lanes)

In [63]:
bike_lane_miles.to_csv(
    CONFIG['clean_dir']
    / f'{INDICATOR}_odp.csv'
)